# Bugdet et commande publique

Ce notebook a pour fonction de proposer une simulton de l’effet qu’aurait un « choc d’inflation » sur les marchés publics conclus par l'État.

Faisant suite à une demande de la directrice du Budget, l'hypothèse retenue pour ce scénario fictif est une hausse de 1 point de l'inflation (IPCH hors tabac) sur 12 mois entre le 1er juillet 2021 et le 30 juin 2022. 

A partir des [données essentielles de la commande publique](https://data.economie.gouv.fr/explore/dataset/decp_augmente/table/) (DECP), publiées par la DAJ du SG Bercy et l'AIFE et augemntées par le BercyHub, il s'agit de proposer une approche par les données de cette simulation. A court terme, cela nécessite de : 

1. Filtrer les données entre État/collectivités territoriales (par exemple sur l'intitulé de l'Acheteur, avec une liste des libellés que l'on garde et une liste des libellés que l'on exclut)
2. Replacer chaque montant sur un référentiel temporaire commun (mois, semestre, ou année ?, et avec les deux types de durées)
3. Calculer l'effet différentiel du choc d'inflation sur les montants totaux

## Sommaire

1. Sur les données de la commande publique
    1. Préparation des données
    2. Étude sur les formes de prix des marchés
        1. Formes de prix des marchés de l'État
        2. Marchés à prix fermes se terminant après juin 2022
    3. Estimation des mensualités de la commande publique
        1. Ensemble des marchés de l'État
        2. Marchés à prix fermes de l'État
2. Sur le budget de l'État
    1. Préparation des données
    2. Filtres
        1. Filtre sur le périmètre ministériel
        2. Filtre sur les comptes budgétaires
        3. Filtre sur les comptes généraux
3. Résutats
    1. Sommes totales
    2. Part des marchés à taux fermes expirant après le 30 juin 2022 dans le total
    3. Exports de travail

In [ ]:
# Librairies employées
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# Suppression du message d'erreur SettingWithCopyWarning
# Cf. https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.set_option('mode.chained_assignment', None)

In [ ]:
# Données sources sur l'ensemble du notebok
decp_safe = pd.read_csv("data/decp_augmente.csv", sep = ";", dtype={'idAcheteur': str, 'montant': float}, encoding="UTF-8")
budget_safe = pd.read_csv('data/ZBUD51 budget Etat 2020.csv', sep = ";", thousands=" ", decimal=",", dtype={'EUR' : float})
filtre_decp_etat_safe = pd.read_csv('data/table corr V2.csv', sep = ";", dtype = {'idAcheteur' : str})
filtre_budget_etat_safe = pd.read_excel('data/filtresPCE.xlsx')
referentiel_ministere_safe = pd.read_excel("data/Copie de RestitCalculReferentiel-20211013_13h13m25s-SUPERADM.xls")

In [ ]:
# Données non utilisées
# p166_safe = pd.read_excel('data/sortieP166.xlsx')

In [ ]:
# Copies de travail des données
decp = decp_safe.copy()
budget = budget_safe.copy()
filtre_decp_etat = filtre_decp_etat_safe.copy()
filtre_budget_etat = filtre_budget_etat_safe.copy()
referentiel_ministere = referentiel_ministere_safe.copy()

## Sur les données de la commande publique

Sur les DECP d'abord, il serait intéressant d'avoir : 

- en nombre et en montant, les formes de prix des marchés (révisable, ferme ou fermes et actualisables) ;
- en nombre et en montant, pour ceux qui ont des prix fermes, ceux qui se terminent après juin 2022 (la temporalité des marchés, notamment leur fin, est à explorer) ;

### Prépartion des données

In [ ]:
# Cette table contient les données essentielles de leurs marchés publics sur leur profil d'acheteur, agrégées et augmentées par le BercyHub
decp.shape

In [ ]:
# On ne conserve que les informations (vraiment) essentielles à ce stade, en excluant notamment la géolocalisation, etc.
keep_cols_decp_work = ['source', 'type', 'natureObjetMarche', 'objetMarche',
       'codeCPV_Original', 'codeCPV', 'codeCPV_division', 'referenceCPV',
       'dateNotification', 'anneeNotification', 'moisNotification',
       'datePublicationDonnees', 'dureeMois', 'dureeMoisEstimee',
       'dureeMoisCalculee', 'montant', 'nombreTitulaireSurMarchePresume',
       'montantCalcule', 'formePrix', 'nature',
       'accord-cadrePresume', 'procedure', 'idAcheteur', 'sirenAcheteurValide',
       'nomAcheteur']
decp_work = decp[keep_cols_decp_work]

In [ ]:
# On ne conserve que les données provenant de l'Agence pour l'information financière de l'État (AIFE)
decp_aife = decp_work[decp_work['source'] == 'data.gouv.fr_aife']
decp_aife.shape

In [ ]:
# On supprime les colonnes booléennes sur la présence d'un libellé de code postal, de commune, département et réguin
keep_cols_filtre_decp_etat = ['idAcheteur', 'nomAcheteur', 'OKKO']
filtre_decp_etat_work = filtre_decp_etat[keep_cols_filtre_decp_etat]

In [ ]:
# Au sein de ces données de l'AIFE, on filtre sur le seul périmètre État, d'après la table établie par 2PERF (DB)
decp_aife_etat = decp_aife.merge(filtre_decp_etat_work, on = ['idAcheteur', 'nomAcheteur'], how = 'inner')
decp_aife_etat = decp_aife_etat[decp_aife_etat['OKKO'] == 'OK']

In [ ]:
# Les commandes publiques de l'État sont essentiellement des marchés
# On identifie quelques contrats de concessions, généralement sans date de notification renseignée
decp_aife_etat.groupby("type").sum()

In [ ]:
# Conversion des dates au format Pandas
decp_aife_etat.loc[:, 'dateNotification'] = pd.to_datetime(decp_aife_etat['dateNotification'], infer_datetime_format=True)

In [ ]:
# On filtre pour les marchés dont la date de notification est postérieure à 2018
decp_aife_etat_post2018 = decp_aife_etat[decp_aife_etat['dateNotification'] > datetime.datetime(2018, 1, 1)]

In [ ]:
# Au sein des données de l'AIFE sur le périmètre État, on garde uniquement les marchés
decp_aife_etat_post2018_marche = decp_aife_etat_post2018[decp_aife_etat_post2018["type"] == "Marché"]

In [ ]:
# Création de la date de fin de contrat calculée, en additionnant la date de notification et le nombre de mois calculé
# Cette information est utile pour l'étude des marchés à prix fermes arrivant  échéance après juin 2022
# elle sert également pour le calcul des mensualités sur le budget
get_dateFinCalculee = lambda x: x['dateNotification'] + relativedelta(months = int(x['dureeMoisCalculee']))
decp_aife_etat_post2018_marche['dateFinCalculee'] = decp_aife_etat_post2018_marche.apply(get_dateFinCalculee, axis=1)

**Point sur la source des données** :

> La date de notification est la date d'attribution du marché ; date de publication est relatif à l'intégration de ces données dans la plateforme. De ce fait, la date de notification n'est pas nécessairement la date de début de contrat

A court terme, on propose une sortie avec les sommes des montants étant terminés en juin 2022, ce qui représente une hypothèse haute, où tous les contrats commençeraient le jour même de leur notification. Une estimation plus précise serait utile dans un second temps, avec, par exemple, le nombre qui y sont encore si l'on décale d'un mois encore, avec une sorte de taux marginal.

### Étude sur les formes de prix des marchés

#### Formes de prix des marchés de l'État

In [ ]:
# Marchés de commande publique sur le périmètre Etat issues des données de l'AIFE dont la date de notification est postérieure à 2018
decp_aife_etat_post2018_marche.shape

In [ ]:
# Calcul, en nombre et en montant, des totaux par formes de prix des marchés
formes_prix = decp_aife_etat_post2018_marche[['formePrix', 'montant', 'montantCalcule']]
formes_prix.groupby(by = "formePrix", axis = 0, dropna=False).agg(['sum','count'])

**Commentaire sur les DECP**: 
    
En matière de type de contrats, l'essentiel des données dont l'on dispose est non-précisée (NaN). La seconde plus importante est celle des prix révisables, alors que les prix fermes représentent un peu moins de la moitité de celui-ci.

In [ ]:
# En sortie, réduction des dimensions au niveau des colonnes
formes_prix.shape

#### Marchés à prix fermes se terminant après juin 2022

In [ ]:
# En entrée, marchés de commande publique sur le périmètre Etat issues des données de l'AIFE dont la date de notification est postérieure à 201
decp_aife_etat_post2018_marche.shape

In [ ]:
# Affichage, en nombre et en montant, pour les marchés à prix fermes, de ceux qui se terminent après juin 2022
prix_fermes = decp_aife_etat_post2018_marche[decp_aife_etat_post2018_marche['formePrix'] == "Ferme"]

In [ ]:
# On filtre pour les marchés finissant après le 30 juin 2022
prix_fermes_juin2022 = prix_fermes[prix_fermes['dateFinCalculee'] > datetime.datetime(2022, 6, 30)]

In [ ]:
# Calcul, en nombre et en montant, de l'hypothèse haute des sommes totales des marchés fermes terminant après juin 2022
prix_fermes_juin2022 = prix_fermes_juin2022[['formePrix', 'montant', 'montantCalcule']]
prix_fermes_juin2022.groupby(by = "formePrix", axis = 0, dropna=False).agg(['sum','count'])

In [ ]:
# En sortie, les marchés de commande publique de l'État à prix fermes, issus des données de l'AIFE, dont la date de notification est postérieure à 2018 
# et dont la date estimée de fin est postérieure au 30 juin 2022
prix_fermes_juin2022.shape

### Estimation des mensualités de la commande publique

#### Ensemble des marchés de l'État

In [ ]:
# Marchés de commande publique sur le périmètre Etat issues des données de l'AIFE dont la date de notification est postérieure à 2018
decp_aife_etat_post2018_marche.shape

In [ ]:
# Construction d'une mensualité théorique estimée
decp_aife_etat_post2018_marche['mensualiteEstimee'] = decp_aife_etat_post2018_marche['montantCalcule'] / decp_aife_etat_post2018_marche['dureeMoisCalculee']

In [ ]:
# Transposition d'un dataframe d'extremum en fréquence
# Source : https://stackoverflow.com/a/57644703
time_series = (decp_aife_etat_post2018_marche[['dateNotification', 'dateFinCalculee']]
               .apply(lambda x: pd.date_range(*x, freq='1M'), # Fréquence mensuelle
                      axis=1)
               .explode()
               .rename('dt') # Nom de l'index
              )

mensualite = decp_aife_etat_post2018_marche.join(time_series).reset_index(drop=True) # jointure sur l'index

In [ ]:
# On conserve la fréquence mensuelle, avec les paiements relatifs à cette dernière
keep_cols_mensualite = ['dt', 'mensualiteEstimee']
mensualite_work = mensualite[keep_cols_mensualite]

In [ ]:
# Regroupement des paiements par an
mensualite_budget_annuel = mensualite_work.groupby(pd.Grouper(key='dt',freq='Y')).count()

#### Marchés à prix fermes de l'État

In [ ]:
prix_fermes.shape

In [ ]:
# Construction d'une mensualité théorique estimée
prix_fermes['mensualiteEstimeePrixFermes'] = prix_fermes['montantCalcule'] / prix_fermes['dureeMoisCalculee']

In [ ]:
# Transposition d'un dataframe d'extremum en fréquence
# Source : https://stackoverflow.com/a/57644703
prix_fermes_explode = (prix_fermes[['dateNotification', 'dateFinCalculee']]
               .apply(lambda x: pd.date_range(*x, freq='1M'), # Fréquence mensuelle
                      axis=1)
               .explode()
               .rename('dt') # Nom de l'index
              )

mensualite_prix_fermes = prix_fermes.join(prix_fermes_explode).reset_index(drop=True) # jointure sur l'index

In [ ]:
# On conserve la fréquence mensuelle, avec les paiements relatifs à cette dernière
keep_cols_mensualite = ['dt', 'mensualiteEstimeePrixFermes']
mensualite_prix_fermes_work = mensualite_prix_fermes[keep_cols_mensualite]

In [ ]:
# Regroupement des paiements par an
mensualite_prix_fermes_budget_annuel = mensualite_prix_fermes_work.groupby(pd.Grouper(key='dt',freq='Y')).sum()

In [ ]:
mensualite_prix_fermes_budget_annuel.shape

#### Comparaisons

Construction d'une table de comparaison, avec la part des paiements des marchés à prix fermes dans l'ensemble des marchés

In [ ]:
mensualite_compare = mensualite_budget_annuel.merge(mensualite_prix_fermes_budget_annuel, on = "dt", how = 'left')

In [ ]:
mensualite_compare['partPrixFermes'] = mensualite_compare['mensualiteEstimeePrixFermes'] / mensualite_compare['mensualiteEstimee']

In [ ]:
mensualite_compare.shape

## Sur le budget de l'État

Sur les dépenses totales de l'État, il faut exclure un certain nombre d'éléments : 

- en termes de ministères (non aux Armées)
- des comptes budgétaires (31, 51 et 52, c'est-à-dire le fonctionnement, les dépenses d’investissement corporels, et incorporels)
- sur le compte général (à voir encore -- par exemple, les fluides, qui sont traités à part). 

### Préparation des données

In [ ]:
# Table contenant l'ensemble des dépenses de l'État pour l'année 2020
budget.shape

budget

In [ ]:
# On ne conserve que les informations pertinentes pour le filtre sur les données
keep_cols_budget_work = ['Compte budgétaire', 'Compte général', 'Référentiel de programmation', 'EUR']
budget_work = budget[keep_cols_budget_work]
budget_work.rename({'Référentiel de programmation' : 'ID_Activite'}, axis = 1, inplace=True)

In [ ]:
budget_work.shape

### Filtres

#### Filtre sur le périmètre ministériel

Il faut exclure le ministère des Armées de la prévision, celui-ci étant pris en compte dans un modèle spécifique.

In [ ]:
# On dispose d'un référentiel des ministères, outil de travail de la DB
print(referentiel_ministere.shape)
print(budget_work.shape)

In [ ]:
# On conserve la colonne de jointure avec les données budgétaires, et le nom du ministère affilié
keep_cols_referentiel_ministere = ['ID_Activite', 'Ministere']
referentiel_ministere_work = referentiel_ministere[keep_cols_referentiel_ministere]
budget_referentiel = budget_work.merge(referentiel_ministere_work, on = "ID_Activite", how = "left") # jointure

In [ ]:
# Exclusion du Ministère des Armées du périmètre
budget_referentiel_sauf_armees = budget_referentiel[budget_referentiel['Ministere'] != "Armées"]

In [ ]:
budget_referentiel_sauf_armees.shape

#### Filtre sur les comptes budgétaires

On conserve les comptes budgétaires (31, 51 et 52, c'est-à-dire le fonctionnement, les dépenses d’investissement corporels, et incorporels)

In [ ]:
keep_comptes_budgetaires = [31, 51, 52]
budget_marches = budget_referentiel_sauf_armees[budget_referentiel_sauf_armees['Compte budgétaire'].isin(keep_comptes_budgetaires)]

#### Filtre sur les comptes généraux

On conserve les comptes identifiés par PERF2 (DB).

In [ ]:
# Comptes à conserver
print(filtre_budget_etat.shape)
print(budget_marches.shape)

In [ ]:
# Renommage pour correspondre aux données budgétaires
filtre_budget_etat.rename({'Compte' : 'Compte général'}, axis=1, inplace=True)

In [ ]:
# Jointure sur la colonne "Compte général"
budget_marches_scope = budget_marches.merge(filtre_budget_etat, on = ['Compte général'], how = 'inner')

In [ ]:
budget_marches_scope.shape

## Comparaison des DECP et de l'exécution budgétaire

### Sommes totales

In [ ]:
# Calcul des sommes totales
tot_budget_marches_scope = budget_marches_scope.EUR.sum()
tot_decp_aife_etat = decp_aife_etat.montantCalcule.sum()
tot_decp_aife_etat_post2018_marche = decp_aife_etat_post2018_marche.montantCalcule.sum()
tot_prix_fermes = prix_fermes.montantCalcule.sum()
tot_prix_fermes_juin2022 = prix_fermes_juin2022.montantCalcule.sum()

In [ ]:
# Impression des résultats
# Affichage en fstring avec 2 chiffres après les unités, point comme indicateur de décimale, virgule pour les séparateurs de milliers
# D'après https://www.peterbe.com/plog/format-thousands-in-python et https://www.python.org/dev/peps/pep-0378/#commentary
print(f"Sur le périmètre défini, la somme totale du budget est de {tot_budget_marches_scope / 1000000:,.2f} millions d'euros")
print(f"Les commandes publiques de l'AIFE sur le périmètre État sont de {tot_decp_aife_etat / 1000000:,.2f} millions d'euros")
print(f"Les marchés de commandes publiques de l'AIFE sur le périmètre État dont la date de notification est postérieure à 2018 sont de {tot_decp_aife_etat_post2018_marche / 1000000:,.2f} millions d'euros")
print(f"Les commandes publiques à taux ferme de l'AIFE sur le périmètre État sont de {tot_prix_fermes / 1000000:,.2f} millions d'euros")
print(f"Les commandes publiques à taux ferme expiration après le 30 juin 2022 de l'AIFE sur le périmètre État sont de {tot_prix_fermes_juin2022 / 1000000:,.2f} millions d'euros")

### Part des marchés à taux fermes expirant après le 30 juin 2022 dans le total

In [ ]:
cols_prix_fermes_juin2022 = [["Dans l'ensemble des marchés à prix fermes de l'État", 100 * (1 - ((tot_prix_fermes - tot_prix_fermes_juin2022) / tot_prix_fermes))], 
        ["Dans l'ensemble des marchés de commandes publiques de l'État postérieures à 2018", 100 * (1 - ((tot_decp_aife_etat_post2018_marche - tot_prix_fermes_juin2022) / tot_decp_aife_etat_post2018_marche))], 
        ["Dans l'ensemble des commandes publiques de l'État", 100 * (1 - ((tot_decp_aife_etat - tot_prix_fermes_juin2022) / tot_decp_aife_etat))], 
        ["Dans le périmètre défini du budget de l'État", 100 * (1 - ((tot_budget_marches_scope - tot_prix_fermes_juin2022) / tot_budget_marches_scope))]]

In [ ]:
proportions_prix_fermes_juin2022 = pd.DataFrame(cols_prix_fermes_juin2022, columns = ['Marchés à taux fermes expirant après le 30 juin 2022', 'Pourcentage'])

In [ ]:
# Contextualisation de l'affichage des nombres dans un dataframe 
# d'après https://stackoverflow.com/a/67187281
with pd.option_context('display.float_format', '{:_.2f}%'.format):
    display(proportions_prix_fermes_juin2022)

### Exports de travail

In [ ]:
# Données sources sur le périmètre des DECP et du budget de l'État
decp_aife_etat_post2018_marche.to_csv("res/decp_aife_etat_post2018_marche.csv", sep = ";",  index=False, decimal=",", float_format='%.3f', encoding="UTF-8")
budget_marches_scope.to_csv("res/budget_marches_scope.csv", sep = ";",  index=False, decimal=",", float_format='%.3f', encoding="UTF-8")

In [ ]:
# FIltres sur les formes de prix des DECP
formes_prix.to_csv("res/formes_prix.csv", sep = ";", index=False, decimal=",", float_format='%.3f', encoding="UTF-8")
prix_fermes_juin2022.to_csv("res/prix_fermes_juin2022.csv", sep = ";",  index=False, decimal=",", float_format='%.3f', encoding="UTF-8")
proportions_prix_fermes_juin2022.to_csv("res/proportions_prix_fermes_juin2022.csv", sep = ";",  index=False, decimal=",", float_format='%.3f', encoding="UTF-8")

In [ ]:
# Mensualisations, et agrégats sur mensualisations
mensualite.to_csv("res/mensualite.csv", sep = ";",  index=False, decimal=",", float_format='%.3f', encoding="UTF-8")
mensualite_budget_annuel.to_csv("res/mensualite_budget_annuel.csv", sep = ";", index_label='annee', decimal=",", float_format='%.3f', encoding="UTF-8")
mensualite_prix_fermes_budget_annuel.to_csv("res/mensualite_prix_fermes_budget_annuel.csv", sep = ";", index_label='annee', decimal=",", float_format='%.3f', encoding="UTF-8")
mensualite_compare.to_csv("res/mensualite_compare.csv", sep = ";", index_label='annee', decimal=",", float_format='%.3f', encoding="UTF-8")